In [50]:
from llama_index.core.llama_dataset import download_llama_dataset
from llama_index.core import VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
import yaml
from pydantic import BaseModel
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextPrecision,
    ContextRecall,
)
from ragas.llms import LlamaIndexLLMWrapper

In [3]:
with open('config.yml') as config_file:
    config_data = yaml.safe_load(config_file)

class DatasetConfig(BaseModel):
    dataset_class: str
    download_dir: str
    
dataset_config = DatasetConfig(**config_data['dataset'])

class LocalModelConfig(BaseModel):
    embedding_model_name: str
    llm_model_name: str

local_model_config = LocalModelConfig(**config_data['local_model'])

In [51]:
llm = Ollama(model=local_model_config.llm_model_name, request_timeout=360.0)

Settings.embed_model = HuggingFaceEmbedding(model_name=local_model_config.embedding_model_name)
Settings.llm = llm
evaluator_llm = LlamaIndexLLMWrapper(llm)

In [5]:
rag_dataset, documents = download_llama_dataset(
    llama_dataset_class=dataset_config.dataset_class, 
    download_dir=dataset_config.download_dir,
    show_progress=True
)

Loading files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:49<00:00,  2.46s/file]


In [6]:
index = VectorStoreIndex.from_documents(
    documents=documents,
    show_progress=True
)

Parsing nodes:   0%|          | 0/1037 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1159 [00:00<?, ?it/s]

In [10]:
query_engine = index.as_query_engine()

In [14]:
predictions = rag_dataset.make_predictions_with(
    predictor = query_engine,
    show_progress = True,
    batch_size = 20
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [01:15<00:00,  5.03s/it]


In [56]:
idx = 0

rag_dataset.examples[idx].query, rag_dataset.examples[idx].reference_answer 

("How has Apple's total net sales changed over time?",
 "Based on the provided documents, Apple's total net sales have changed over time as follows:\n\n- For the quarterly period ended June 25, 2022, the total net sales were $82,959 million. (SOURCE: 2022 Q3 AAPL.pdf)\n- For the quarterly period ended December 31, 2022, the total net sales were $117,154 million. (SOURCE: 2023 Q1 AAPL.pdf)\n- For the quarterly period ended April 1, 2023, the total net sales were $94,836 million. (SOURCE: 2023 Q2 AAPL.pdf)\n- For the quarterly period ended July 1, 2023, the total net sales were $81,797 million. (SOURCE: 2023 Q3 AAPL.pdf)\n\nFrom these figures, it can be observed that there was an increase in total net sales from the quarter ended June 25, 2022, to the quarter ended December 31, 2022. However, there was a subsequent decrease in total net sales in the quarters ended April 1, 2023, and July 1, 2023.\n\nSOURCE(S): 2022 Q3 AAPL.pdf, 2023 Q1 AAPL.pdf, 2023 Q2 AAPL.pdf, 2023 Q3 AAPL.pdf")

In [60]:
predictions.predictions[idx].response, predictions.predictions[idx].contexts

("Over time, Apple's total net sales have decreased.",
 ['Products and Services Performance\nThe following table shows net sales by category for the three- and six-month periods ended April 1, 2023 and March 26, 2022 (dollars in millions):\nThree Months Ended Six Months Ended\nApril 1,2023 March 26,2022 Change April 1,2023 March 26,2022 Change\nNet sales by category:\niPhone $ 51,334 $ 50,570 2 % $ 117,109 $ 122,198 (4)%\nMac 7,168 10,435 (31)% 14,903 21,287 (30)%\niPad 6,670 7,646 (13)% 16,066 14,894 8 %\nWearables, Home and Accessories 8,757 8,806 (1)% 22,239 23,507 (5)%\nServices 20,907 19,821 5 % 41,673 39,337 6 %\nTotal net sales $ 94,836 $ 97,278 (3)% $ 211,990 $ 221,223 (4)%\niPhone\niPhone net sales were relatively flat during the second quarter of 2023 compared to the second quarter of 2022. Year-over-year iPhone net sales decreasedduring the first six months of 2023 due primarily to lower net sales from the Company’s new iPhone models launched in the fourth quarter of 2022.\n